In [ ]:
# Uncomment and run this when using colab. 
#%pip install pygeoinf

# Tutorial 2 - Hilbert spaces

Hilbert spaces are complete inner product spaces. Within all inverse problems considered we will assume that both the model and data space are Hilbert spaces. This is a restriction, but one that is difficult to avoid in practice. The data space in any real inverse problem is necessarily finite dimensional and so can always be identified with $\mathbb{R}^{n}$ for some 
suitable $n \in \mathbb{N}$. In most cases of interest, however, the model space will be comprised of functions and be infinite-dimensional. Computationally we cannot deal directly with infinite-dimensional spaces, but we can ensure that we work with convergent discretisations. This means, roughly speaking, that the underlying mathematical problems are well-defined, and that the numerical methods can be shown to converge to the exact values as the size of the discretised model space is suitably increased. A principal aim of ```pygeoinf``` is to facilitate the implementation of such methods in the context of geophysical inverse problems. 

In this tutorial we look into the representation of Hilbert spaces within the ```pygeoinf``` library through two examples. 
Hilbert spaces are modeled by the ```HilbertSpace``` class. Importantly this class defines only the *structure* of the Hilbert space but not its elements. To construct a class instance the user must provide the following data:

- The dimension of the space. 
- A mapping from elements of the space to a chosen co-ordinate representation. 
- A mapping from this co-ordinate representation to elements of the space. 
- The inner product for the space. 
- A mapping from elements of the space to their associated dual vector.
- A mapping from elements of the dual space into the their representation within the space. 

In the case of infinite-dimensional spaces, the dimension, $n$, is understood as being that of an approximating space that is isomorphic to $\mathbb{R}^{n}$  through the co-ordinate mappings provided. 

Discussion of the final two points in the above list will be given in the following tutorial once Linear forms and dual spaces have been introduced.  


## Euclidean space

The simplest Hilbert space is Euclidean space. This is implemented in ```pygeoinf``` as ```EuclideanSpace``` that inherits from ```HilbertSpace```. Constructuion of an instance of this class requires only its dimension: 

In [ ]:
import numpy as np
import pygeoinf as inf

# Set up Euclidean space. 
dim = 4
X = inf.EuclideanSpace(dim)

# Print its dimension. 
print(f'Dimension of the vector space = {X.dim}')

Elements of ```EuclideanSpace``` are just ```numpy``` vectors. In the following code block we generate a random element and get its components. For ```EuclideanSpace```, a vector and the vector of its components are the same and so the ```to_component``` and ```from_component``` methods are trivial. 

In [ ]:
# Generate a random element of the space and print its value. 
x = X.random()
print(f'The vector is equal to:               {x}')

# Now get its component vector which, in this case, is the same thing. 
c = X.to_components(x)
print(f'The vector\'s components are equal to: {c}')

Note that the ```random``` method for ```HilbertSpaces``` internally generates a component vector drawn from a standard Gaussian distribution and then maps this to an element of the space. This method is only meant to provide a quick means of generating elements of the space for testing purposes. 

If needed, we can access the zero vector is the space as follows:

In [ ]:
# Generate a the zero vector -- note that this is a property of the class. 
x = X.zero
print(f'The vector is equal to:               {x}')

# Now get its component vector, which in this case is the same thing. 
c = X.to_components(x)
print(f'The vector\'s components are equal to: {c}')

Given two element of the space, we can compute their inner product:

In [ ]:
x1 = X.random()
x2 = X.random()
print(f'x1       =  {x1} ')
print(f'x2       =  {x2} ')

print(f'(x1, x2) =  {X.inner_product(x1, x2)}')

# Check the inner product is symmetric.
print(f'(x2, x1) =  {X.inner_product(x2, x1)}')

The inner product defines a norm, and we can access this as follows:

In [ ]:
x = X.random()
print(f'x       =  {x}')
print(f'||x||   =  {X.norm(x)}')
print(f'||x||^2 =  {X.squared_norm(x)}')

We can use these methods to verify that standard Hilbert space inequalities are satified. 

In [ ]:
x1 = X.random()
x2 = X.random()
print(f"x1 =  {x1} ")
print(f"x2 =  {x2} ")

print(
    f"The triangle inequality, |x1 + x2| <= |x1| + |x2|, is {X.norm(x1+x2) <= X.norm(x1) + X.norm(x2)}."
)

print(
    f"The Cauchy-Scwharz inequality, |(x1,x2)| <= |x1||x2|, is {np.abs(X.inner_product(x1,x2)) <= X.norm(x1) * X.norm(x2)}."
)


For ```HilbertSpaces``` whose elements have the standard vector operations already overloaded we can directly add, subtract or scalar multiply them. 

Such operations are also available through methods within the ```HilbertSpace``` class. For example

In [ ]:
x1 = X.random()
x2 = X.random()  
print(f"x1      = {x1} ")
print(f"x2      = {x2} ")

x3 = X.add(x1,x2)
print(f"x1 + x2 = {x3} ")

print(f"2 * x1  = {X.multiply(2,x1)} ")

# Form a deep copy of x3
x4 = X.copy(x3)
print(f"x4      = {x4} ")



Custom implementation for these operations can be provided when a ```HilbertSpace``` is constructed. 

The ```HilbertSpace``` class also provides convenient methods for generating certain element of the space. For example, if we want a particular basis vector we write:

In [ ]:
i = 2
x = X.basis_vector(2)
print(f"The {i}th basis vector is: {x} ")

## Sobolev spaces on the unit circle 

### Mathematical definitions

 
Within the example inverse problem in Tutorial 1, the data space was represented by an instance of ```EuclideanSpace```. The model space, however, provides a more interesting example of a Hilbert space, $H^{s}(\mathbb{S}^{1})$, that is defined by
$$
H^{s}(\mathbb{S}^{1}) = \{
u \in \mathcal{D}'(\mathbb{S}^{1}) \,|\, (1+\lambda^{2}\Delta)^{s/2} u \in L^{2}(\mathbb{S}^{1})
\},
$$
where the **order**, $s\in \mathbb{R}$, and **scale** $\lambda > 0$. Here $\mathcal{D}'(\mathbb{S}^{1})$ is the space of distributions on $\mathbb{S}^{1}$, $L^{2}(\mathbb{S}^{2})$ the Hilbert space of square-integrable functions that has inner product
$$
(u,v)_{L^{2}(\mathbb{S}^{1})} = \int_{0}^{2\pi} \overline{u} v \, \mathrm{d} \theta, 
$$
and $\Delta = -\frac{\mathrm{d}^{2}}{\mathrm{d}\theta^{2}}$ is the Laplacian. The latter operator extends to a non-negative self-adjoint operator on $L^{2}(\mathbb{S}^{1})$ and $(1+\lambda^{2}\Delta)^{s/2}$ is defined through the spectral theorem. The inner product on this space is given by
$$
(u,v)_{H^{s}(\mathbb{S}^{1})} = \left((1+\lambda^{2}\Delta)^{s/2} u,(1+\lambda^{2}\Delta)^{s/2} v\right)_{L^{2}(\mathbb{S}^{1})}.
$$
Clearly we have $H^{0}(\mathbb{S}^{1}) = L^{2}(\mathbb{S}^{1})$, while for any $t > s$ we have the  continuous and compact inclusion
$$
H^{t}(\mathbb{S}^{1}) \subseteq H^{s}(\mathbb{S}^{1}).
$$
Note that the topology of $H^{s}(\mathbb{S}^{1})$ is independent of the scale. 

To work with this space practically, we can use Fourier decomposition. For any $u \in \mathcal{D}'(\mathbb{S}^{1})$ we have the convergent Fourier expansion
$$
u = \sum_{k=-\infty}^{\infty} u_{k} \phi_{k}, \quad u_{k} = (\phi_{k},u)_{L^{2}(\mathbb{S}^{1})},
$$
where $L^{2}(\mathbb{S}^{1}) \subseteq \mathcal{D}'(\mathbb{S}^{1})$ in the standard way, and the basis functions
$$
\phi_{k}(\theta) = \frac{1}{\sqrt{2\pi}} \mathrm{e}^{\mathrm{i} k \theta}
$$
are complete and orthonormal in $L^{2}(\mathbb{S}^{1})$. It can be shown that such a $u \in \mathcal{D}'(\mathbb{S}^{1})$ belongs to $L^{2}(\mathbb{S}^{1})$ if and only if its Fourier coefficients are square-summable. It follows that we can equivalently characterise elements of $H^{s}(\mathbb{S}^{1})$ through
$$
\sum_{k=-\infty}^{\infty}(1 + \lambda^{2}k^{2})^{s}|u_{k}|^{2} < \infty, 
$$
while the space's inner product can be written 
$$
(u,v)_{H^{s}(\mathbb{S}^{1})} = \sum_{k=-\infty}^{\infty}(1 + \lambda^{2}k^{2})^{s} \overline{u_{k}} v_{k}.
$$
In the above discussion we have considered complex-valued functions for convenience. The definitions all specialise to the real case where we have the symmetry $\overline{u_{k}}  = u_{-k}$ for the Fourier coefficients. 

### Implementation as a ```HilbertSpace```

An implementation of $H^{1}(\mathbb{S}^{1})$ can be found in the class ```Sobolev``` which inherits from ```HilbertSpace``` and is contained in the sub-package ```pygeoinf.symmetric_space.circle```. Elements of the space are numpy vectors of function values on an equally spaced grid. The components are then Fourier coefficients for non-negative indices, with the mapping from a vector to its components performed using a real FFT. To construct an instance of this space, the user provides:

- A maximum degree, $k_{\mathrm{max}} \in \mathbb{N}$, for the Fourier expansions. The number of spatial points is equal to twice this value. 
- The Sobolev order, $s \in \mathbb{R}$. 
- The Sobolev scale, $\lambda >0$. 
- An optional radius, $a > 0$, for the circle with default value $a = 1$. When $a \ne 1$ the above definitions are transformed in the obvious manner. 

The code below constructs an instance of the ``Sobolev``` class and prints its dimension:

In [ ]:
from pygeoinf.symmetric_space.circle import Sobolev
kmax = 245
order = 2
scale = 0.1
Y = Sobolev(kmax, order, scale)
print(f'The dimension of the space is equal to: {Y.dim}')

We can then visualise the first few basis vectors as follows:

In [ ]:
import matplotlib.pyplot as plt

m = 4
fig, axs = plt.subplots(m,m, figsize=(15,15))

for j in range(m):
    for i in range(m):
        ax = axs[i,j]
        k = i + m * j
        u = Y.basis_vector(k)
        Y.plot(u, fig=fig, ax=ax)
        ax.set_title(f'Basis function {k}')

plt.tight_layout()
plt.show()

Note that each of the basis vector shown is a cosine function. This is because the real and imaginary parts of the Fourier coefficients are separated out in the component vector, with the real parts being stored first. 

Elements of this space can also be defined by projecting a given function:

In [ ]:
u = Y.project_function(lambda th: np.sin(th)*np.cos(th))
Y.plot(u)
plt.show()

All the vector operations work on this space just as with ```EuclideanSpace```. For example:

In [ ]:
v = u +  Y.project_function(lambda th: np.cos(5*th))
print(f'||u|| = {Y.norm(u)}')
print(f'||v|| =  {Y.norm(v)}')
print(f'(u,v) =  {Y.inner_product(u,v)}')

The appropriate choice of $k_{\mathrm{max}}$ in the construction of the ```Sobolev``` class depends on the application. For convenience, a static method is provided that determines an generally sensible choice of $k_{\mathrm{max}}$ based on the Sobolev parameters. This is shown below:

In [ ]:
Y = Sobolev.from_sobolev_parameters(2, 0.1)
print(f'Dimension of space = {Y.dim}')

 The user can optionally provide a relative tolerance through the keyword argument ```rtol``` and ensure that $k_{\mathrm{max}}$ is a power of 2 by setting the key-word argument ```power_of_two``` equal to ```True```. Note, however, that this method is only available when $s > 0.5$.